In [1]:
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt
from varclushi import VarClusHi
from sklearn.metrics import silhouette_score

from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
import keras
from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
import warnings

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
# import keras
import math

from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from methods import *
import warnings
from tensorflow.keras import optimizers
warnings.filterwarnings("ignore")
%matplotlib inline

warnings.filterwarnings('ignore')
print("okay")


okay


In [2]:
df = pd.DataFrame()
training_path = 'PRONOSTIA/Learning_set'
bearing = 'Bearing1_1'
base_dir = os.path.join(training_path, bearing)

In [3]:
def load_vibration_data(data_folder):
    """Charge les fichiers de vibration et calcule le temps en secondes pour chaque échantillon."""
    vibration_files = [f for f in os.listdir(data_folder) if f.startswith('acc_') and f.endswith('.csv')]
    vibration_data = []

    for file in vibration_files:
        file_path = os.path.join(data_folder, file)
        df = pd.read_csv(file_path, names=['Hour', 'Minute', 'Second', 'Microsecond', 'Horizontal_Accel', 'Vertical_Accel'])
        
        # Extraction de l'identifiant du roulement
        bearing_id = file.split('_')[1].split('.')[0]
        df['bearing_id'] = bearing_id
        
        # Calcul du temps en secondes pour chaque échantillon
        df['time_seconds'] = df['Hour'] * 3600 + df['Minute'] * 60 + df['Second'] + df['Microsecond'] * 1e-6
        vibration_data.append(df)

    # Concaténation de toutes les données de vibration en un DataFrame unique
    return pd.concat(vibration_data, ignore_index=True)

def calculate_total_time(vibration_data):
    """Calcule le temps total de l'expérience pour chaque roulement."""
    return vibration_data.groupby('bearing_id')['time_seconds'].max()

def calculate_rul(vibration_data, total_time):
    """Calcule la RUL pour chaque échantillon en utilisant la formule RUL = (tend - tsample) / tend."""
    vibration_data['RUL'] = vibration_data.apply(
        lambda row: (total_time[row['bearing_id']] - row['time_seconds']) / total_time[row['bearing_id']]*100000,
        axis=1
    )
    return vibration_data

def main(data_folder):
    """Fonction principale pour charger les données, calculer le temps total et la RUL."""
    # Charger les données de vibration
    vibration_data = load_vibration_data(data_folder)
    
    # Calculer le temps total de l'expérience pour chaque roulement
    total_time = calculate_total_time(vibration_data)
    
    # Calculer la RUL
    vibration_data_with_rul = calculate_rul(vibration_data, total_time)
    
    return vibration_data_with_rul

# Utilisation de la fonction principale
data_folder = base_dir
vibration_data_with_rul = main(data_folder)
print(vibration_data_with_rul[['bearing_id', 'time_seconds', 'RUL']].head())


  bearing_id  time_seconds       RUL
0      00001  34779.065664  0.287402
1      00001  34779.065703  0.287290
2      00001  34779.065742  0.287178
3      00001  34779.065781  0.287066
4      00001  34779.065820  0.286953


In [4]:
import pandas as pd
import os

def load_vibration_data(data_folder):
    """Charge les fichiers de vibration et calcule le temps en secondes pour chaque échantillon."""
    vibration_files = [f for f in os.listdir(data_folder) if f.startswith('acc_') and f.endswith('.csv')]
    vibration_data = []

    for file in vibration_files:
        file_path = os.path.join(data_folder, file)
        df = pd.read_csv(file_path, names=['Hour', 'Minute', 'Second', 'Microsecond', 'Horizontal_Accel', 'Vertical_Accel'])
        
        # Extraction de l'identifiant du roulement
        bearing_id = file.split('_')[1].split('.')[0]
        df['bearing_id'] = bearing_id
        
        # Calcul du temps en secondes pour chaque échantillon
        df['time_seconds'] = df['Hour'] * 3600 + df['Minute'] * 60 + df['Second'] + df['Microsecond'] * 1e-6
        vibration_data.append(df)

    # Concaténation de toutes les données de vibration en un DataFrame unique
    return pd.concat(vibration_data, ignore_index=True)

def calculate_total_time(vibration_data):
    """Calcule le temps total de l'expérience pour chaque roulement."""
    return vibration_data.groupby('bearing_id')['time_seconds'].max()

def calculate_normalized_rul(vibration_data, total_time):
    """Calcule le RUL normalisé pour chaque échantillon en utilisant la formule RUL_norm = 1 - (RUL_t / T_cycle)."""
    vibration_data['RUL_norm'] = vibration_data.apply(
        lambda row: 1 - (total_time[row['bearing_id']] - row['time_seconds']) / total_time[row['bearing_id']],
        axis=1
    )
    return vibration_data

def main(data_folder):
    """Fonction principale pour charger les données, calculer le temps total et le RUL normalisé."""
    # Charger les données de vibration
    vibration_data = load_vibration_data(data_folder)
    
    # Calculer le temps total de l'expérience pour chaque roulement
    total_time = calculate_total_time(vibration_data)
    
    # Calculer le RUL normalisé
    vibration_data_with_rul = calculate_normalized_rul(vibration_data, total_time)
    
    return vibration_data_with_rul

# Utilisation de la fonction principale
# data_folder = 'chemin/vers/dossier_donnees'
vibration_data_with_rul = main(data_folder)
print(vibration_data_with_rul[['bearing_id', 'time_seconds', 'RUL_norm']].head())

  bearing_id  time_seconds  RUL_norm
0      00001  34779.065664  0.999997
1      00001  34779.065703  0.999997
2      00001  34779.065742  0.999997
3      00001  34779.065781  0.999997
4      00001  34779.065820  0.999997


In [5]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis, skew
from scipy.fft import fft
import os

def load_vibration_data(data_folder):
    """Charge les fichiers de vibration et calcule le temps en secondes pour chaque échantillon."""
    vibration_files = [f for f in os.listdir(data_folder) if f.startswith('acc_') and f.endswith('.csv')]
    vibration_data = []

    for file in vibration_files:
        file_path = os.path.join(data_folder, file)
        df = pd.read_csv(file_path, names=['Hour', 'Minute', 'Second', 'Microsecond', 'Horizontal_Accel', 'Vertical_Accel'])
        
        # Extraction de l'identifiant du roulement
        bearing_id = file.split('_')[1].split('.')[0]
        df['bearing_id'] = bearing_id
        
        # Calcul du temps en secondes pour chaque échantillon
        df['time_seconds'] = df['Hour'] * 3600 + df['Minute'] * 60 + df['Second'] + df['Microsecond'] * 1e-6
        vibration_data.append(df)

    # Concaténation de toutes les données de vibration en un DataFrame unique
    return pd.concat(vibration_data, ignore_index=True)

# Caractéristiques temporelles
def extract_temporal_features(signal):
    """Extrait les caractéristiques temporelles : moyenne, écart type, RMS, skewness, etc."""
    features = {
        'mean': signal.mean(),
        'std': signal.std(),
        'peak_to_peak': signal.max() - signal.min(),
        'rms': np.sqrt(np.mean(signal**2)),
        'mean_abs': np.mean(np.abs(signal)),
        'max_abs': np.max(np.abs(signal)),
        'skewness': skew(signal),
        'kurtosis': kurtosis(signal),
        'form_factor': np.sqrt(np.mean(signal**2)) / np.mean(np.abs(signal)),
        'crest_factor': signal.max() / np.sqrt(np.mean(signal**2)),
        'impulse_factor': signal.max() / np.mean(np.abs(signal)),
        'margin_factor': signal.max() / (np.mean(np.sqrt(np.abs(signal))) ** 2)
    }
    return features

# Caractéristiques fréquentielles
def extract_frequency_features(signal, sampling_rate=25600):
    """Extrait les caractéristiques fréquentielles."""
    N = len(signal)
    freqs = fft(signal)
    freqs = np.abs(freqs[:N // 2])  # On garde seulement les composantes positives
    freq_bins = np.fft.fftfreq(N, d=1/sampling_rate)[:N // 2]

    max_amplitude = np.max(freqs)
    energy_band1 = np.sum(freqs[(freq_bins >= 0) & (freq_bins < 500)]**2)
    energy_band2 = np.sum(freqs[(freq_bins >= 500) & (freq_bins < 1000)]**2)
    mean_freq = np.mean(freqs)
    rms_freq = np.sqrt(np.mean(freqs**2))
    
    features = {
        'max_amplitude': max_amplitude,
        'energy_band1': energy_band1,
        'energy_band2': energy_band2,
        'mean_freq': mean_freq,
        'rms_freq': rms_freq,
        'variance_freq': np.var(freqs),
        'std_freq': np.std(freqs),
        'kurtosis_freq': kurtosis(freqs),
        'skewness_freq': skew(freqs),
        'peak_freq': freqs.max(),
        'form_factor_freq': rms_freq / mean_freq,
        'crest_factor_freq': max_amplitude / rms_freq
    }
    return features

def extract_all_features(data_folder):
    """Fonction principale pour extraire les 52 caractéristiques (26 pour chaque direction de vibration)."""
    # Charger les données de vibration
    vibration_data = load_vibration_data(data_folder)
    
    # Initialiser un DataFrame pour stocker toutes les caractéristiques
    features_list = []

    # Extraire les caractéristiques pour chaque roulement
    for bearing_id, group in vibration_data.groupby('bearing_id'):
        temporal_features_h = extract_temporal_features(group['Horizontal_Accel'])
        temporal_features_v = extract_temporal_features(group['Vertical_Accel'])
        frequency_features_h = extract_frequency_features(group['Horizontal_Accel'])
        frequency_features_v = extract_frequency_features(group['Vertical_Accel'])
        
        # Renommer les caractéristiques pour chaque direction
        features = {f"{k}_h": v for k, v in temporal_features_h.items()}
        features.update({f"{k}_v": v for k, v in temporal_features_v.items()})
        features.update({f"{k}_freq_h": v for k, v in frequency_features_h.items()})
        features.update({f"{k}_freq_v": v for k, v in frequency_features_v.items()})
        
        # Ajouter l'identifiant du roulement
        features['bearing_id'] = bearing_id
        
        # Ajouter aux résultats
        features_list.append(features)

    # Convertir la liste de caractéristiques en DataFrame
    features_df = pd.DataFrame(features_list)
    return features_df

# Utilisation de la fonction principale
# data_folder = 'chemin/vers/dossier_donnees'

In [9]:
features_df = extract_all_features(data_folder)
features_df.shape

(2803, 45)

In [10]:
def extract_frequency_features(signal, sampling_rate=25600):
    """Extrait les caractéristiques fréquentielles."""
    if signal.empty:
        print("Signal vide. Aucun calcul de caractéristiques fréquentielles.")
        return {}
    
    # Vérifier que le signal est sous forme numérique
    signal = signal.dropna()
    if not np.issubdtype(signal.dtype, np.number):
        print("Le signal contient des données non numériques.")
        return {}

    try:
        N = len(signal)
        freqs = fft(signal)
        freqs = np.abs(freqs[:N // 2])  # On garde seulement les composantes positives
        freq_bins = np.fft.fftfreq(N, d=1/sampling_rate)[:N // 2]

        max_amplitude = np.max(freqs)
        energy_band1 = np.sum(freqs[(freq_bins >= 0) & (freq_bins < 500)]**2)
        energy_band2 = np.sum(freqs[(freq_bins >= 500) & (freq_bins < 1000)]**2)
        mean_freq = np.mean(freqs)
        rms_freq = np.sqrt(np.mean(freqs**2))
        
        features = {
            'max_amplitude': max_amplitude,
            'energy_band1': energy_band1,
            'energy_band2': energy_band2,
            'mean_freq': mean_freq,
            'rms_freq': rms_freq,
            'variance_freq': np.var(freqs),
            'std_freq': np.std(freqs),
            'kurtosis_freq': kurtosis(freqs),
            'skewness_freq': skew(freqs),
            'peak_freq': freqs.max(),
            'form_factor_freq': rms_freq / mean_freq,
            'crest_factor_freq': max_amplitude / rms_freq
        }
        return features
    
    except KeyError as e:
        print(f"Erreur dans l'extraction des caractéristiques fréquentielles : {e}")
        return {}


In [11]:
import numpy as np
from scipy.fft import fft
from scipy.stats import kurtosis, skew

def extract_frequency_features(signal, sampling_rate=25600):
    """Extrait les caractéristiques fréquentielles d'un signal donné."""
    # Convertir le signal en tableau NumPy
    signal = np.asarray(signal)

    # Vérifier si le signal est vide après conversion
    if signal.size == 0:
        raise ValueError("Signal vide : impossible de calculer des caractéristiques fréquentielles.")

    # Appliquer la FFT
    N = len(signal)
    freqs = fft(signal)
    freqs = np.abs(freqs[:N // 2])  # On garde seulement les composantes positives
    freq_bins = np.fft.fftfreq(N, d=1/sampling_rate)[:N // 2]
    
    # Calcul des caractéristiques fréquentielles
    max_amplitude = np.max(freqs)
    mean_freq = np.mean(freqs)
    rms_freq = np.sqrt(np.mean(freqs**2))
    variance_freq = np.var(freqs)
    std_freq = np.std(freqs)
    kurtosis_freq = kurtosis(freqs)
    skewness_freq = skew(freqs)
    peak_freq = freq_bins[np.argmax(freqs)]
    form_factor_freq = rms_freq / mean_freq if mean_freq != 0 else 0
    crest_factor_freq = max_amplitude / rms_freq if rms_freq != 0 else 0

    features = {
        'max_amplitude': max_amplitude,
        'mean_freq': mean_freq,
        'rms_freq': rms_freq,
        'variance_freq': variance_freq,
        'std_freq': std_freq,
        'kurtosis_freq': kurtosis_freq,
        'skewness_freq': skewness_freq,
        'peak_freq': peak_freq,
        'form_factor_freq': form_factor_freq,
        'crest_factor_freq': crest_factor_freq
    }
    return features
